<a href="https://colab.research.google.com/github/data-aleks/henley_passport_index_tidy_tuesday_09_09_09/blob/main/tidytuesday_09092025_henley_passport_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
import pandas as pd
import ast

#**Henley Passport Index Data - Tidy Tuesday - 09.09.09**

his week we are exploring data from the Henley Passport Index API. The Henley Passport Index is produced by Henley & Partners and captures the number of countries to which travelers in possession of each passport in the world may enter visa free.

For each travel destination, if no visa is required for passport holders from a country or territory, then a score with value = 1 is created for that passport. A score with value = 1 is also applied if passport holders can obtain a visa on arrival, a visitor’s permit, or an electronic travel authority (ETA) when entering the destination. These visa-types require no pre-departure government approval, because of the specific visa-waiver programs in place. Where a visa is required, or where a passport holder has to obtain a government-approved electronic visa (e-Visa) before departure, a score with value = 0 is assigned. A score with value = 0 is also assigned if passport holders need pre-departure government approval for a visa on arrival, a scenario we do not consider ‘visa-free’. The total score for each passport is equal to the number of destinations for which no visa is required (value = 1), under the conditions defined above.

Henley & Partners update the Global Passport Index rankings each month and changes to the US passport rank captured media attention recently.

- Which countries have made the most dramatic improvements in passport power?
- Which passports have lost the most visa-free access?
- Do countries with stronger trade relationships tend to offer mutual visa-free access?
- Did the COVID-19 pandemic affect passport rankings, particularly for countries that implemented strict border controls?
- How does political instability or economic crisis impact passport strength?

Thank you to [Brenden Smith and Jen Richmond](https://github.com/brendensm @jenrichmond) for curating this week's dataset.

**Extra data not provided in the challenge.**

To answer some of the stated questions in the dataset we will have to find some external data, as the data provided with the challenge has no covid-19 related data or any economic data.

**So here are some of the external data sources i will utilise**
- [Oxford Covid-19 Government Response Tracker](https://github.com/OxCGRT/covid-policy-dataset/tree/main) I will use this data to attempt to answer covid related question.

## **Step 1** Loading the data

In [86]:
df_country_list = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/main/data/2025/2025-09-09/country_lists.csv')
df_rank_by_year = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/main/data/2025/2025-09-09/rank_by_year.csv')
df_covid_data = pd.read_csv('https://raw.githubusercontent.com/OxCGRT/covid-policy-dataset/refs/heads/main/data/OxCGRT_compact_national_v1.csv')

In [87]:
if df_country_list.empty or df_rank_by_year.empty or df_covid_data.empty:
  print('Issue with data, check source.')
else:
  # Save the initial datasets
  df_country_list.to_csv('df_country_list.csv', index=False)
  df_rank_by_year.to_csv('df_rank_by_year.csv', index=False)
  print('Data Loaded succesfully.')

Data Loaded succesfully.


## **Step 2** Let's have a look at how our data looks.

### **Step 2.1** Country list dataset

In this step i will look at the dataset to identify any cleaning, organisational steps to be taken in data cleaning step.

In [88]:
df_country_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 7 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   code                             198 non-null    object
 1   country                          199 non-null    object
 2   visa_required                    199 non-null    object
 3   visa_online                      199 non-null    object
 4   visa_on_arrival                  199 non-null    object
 5   visa_free_access                 199 non-null    object
 6   electronic_travel_authorisation  199 non-null    object
dtypes: object(7)
memory usage: 11.0+ KB


In [89]:
df_country_list.isna().sum()

,0
code,1
country,0
visa_required,0
visa_online,0
visa_on_arrival,0
visa_free_access,0
electronic_travel_authorisation,0


In [90]:
df_country_list[df_country_list['code'].isna()]

,code,country,visa_required,visa_online,visa_on_arrival,visa_free_access,electronic_travel_authorisation
80,NaN,Namibia,"[[{""code"":""AF"",""name"":""Afghanistan""},{""code"":""...","[[{""code"":""AZ"",""name"":""Azerbaijan""},{""code"":""C...","[[{""code"":""BD"",""name"":""Bangladesh""},{""code"":""B...","[[{""code"":""AO"",""name"":""Angola""},{""code"":""AI"",""...","[[{""code"":""LK"",""name"":""Sri Lanka""},{""code"":""SC..."


In [91]:
df_country_list.head()

,code,country,visa_required,visa_online,visa_on_arrival,visa_free_access,electronic_travel_authorisation
0,PS,Palestinian Territory,"[[{""code"":""AF"",""name"":""Afghanistan""},{""code"":""...","[[{""code"":""AG"",""name"":""Antigua and Barbuda""},{...","[[{""code"":""BD"",""name"":""Bangladesh""},{""code"":""B...","[[{""code"":""BO"",""name"":""Bolivia""},{""code"":""CK"",...","[[{""code"":""LK"",""name"":""Sri Lanka""},{""code"":""KE..."
1,AD,Andorra,"[[{""code"":""AF"",""name"":""Afghanistan""},{""code"":""...","[[{""code"":""AO"",""name"":""Angola""},{""code"":""AZ"",""...","[[{""code"":""BH"",""name"":""Bahrain""},{""code"":""BD"",...","[[{""code"":""JP"",""name"":""Japan""},{""code"":""AL"",""n...","[[{""code"":""AU"",""name"":""Australia""},{""code"":""CA..."
2,VA,Vatican City,"[[{""code"":""AF"",""name"":""Afghanistan""},{""code"":""...","[[{""code"":""AZ"",""name"":""Azerbaijan""},{""code"":""B...","[[{""code"":""BH"",""name"":""Bahrain""},{""code"":""BD"",...","[[{""code"":""AL"",""name"":""Albania""},{""code"":""AD"",...","[[{""code"":""AU"",""name"":""Australia""},{""code"":""CA..."
3,SM,San Marino,"[[{""code"":""AF"",""name"":""Afghanistan""},{""code"":""...","[[{""code"":""AZ"",""name"":""Azerbaijan""},{""code"":""B...","[[{""code"":""BH"",""name"":""Bahrain""},{""code"":""BD"",...","[[{""code"":""JP"",""name"":""Japan""},{""code"":""AL"",""n...","[[{""code"":""AU"",""name"":""Australia""},{""code"":""CA..."
4,MC,Monaco,"[[{""code"":""AF"",""name"":""Afghanistan""},{""code"":""...","[[{""code"":""AZ"",""name"":""Azerbaijan""},{""code"":""B...","[[{""code"":""BH"",""name"":""Bahrain""},{""code"":""BD"",...","[[{""code"":""JP"",""name"":""Japan""},{""code"":""AL"",""n...","[[{""code"":""AU"",""name"":""Australia""},{""code"":""CA..."


In [92]:
df_country_list.tail()

,code,country,visa_required,visa_online,visa_on_arrival,visa_free_access,electronic_travel_authorisation
194,AG,Antigua and Barbuda,"[[{""code"":""AF"",""name"":""Afghanistan""},{""code"":""...","[[{""code"":""AZ"",""name"":""Azerbaijan""},{""code"":""B...","[[{""code"":""AM"",""name"":""Armenia""},{""code"":""BD"",...","[[{""code"":""AL"",""name"":""Albania""},{""code"":""AD"",...","[[{""code"":""LK"",""name"":""Sri Lanka""},{""code"":""KR..."
195,AO,Angola,"[[{""code"":""AF"",""name"":""Afghanistan""},{""code"":""...","[[{""code"":""AG"",""name"":""Antigua and Barbuda""},{...","[[{""code"":""BD"",""name"":""Bangladesh""},{""code"":""B...","[[{""code"":""BJ"",""name"":""Benin""},{""code"":""BW"",""n...","[[{""code"":""LK"",""name"":""Sri Lanka""},{""code"":""SC..."
196,DZ,Algeria,"[[{""code"":""AF"",""name"":""Afghanistan""},{""code"":""...","[[{""code"":""AG"",""name"":""Antigua and Barbuda""},{...","[[{""code"":""BO"",""name"":""Bolivia""},{""code"":""BI"",...","[[{""code"":""BJ"",""name"":""Benin""},{""code"":""CK"",""n...","[[{""code"":""LK"",""name"":""Sri Lanka""},{""code"":""SC..."
197,AL,Albania,"[[{""code"":""AF"",""name"":""Afghanistan""},{""code"":""...","[[{""code"":""BJ"",""name"":""Benin""},{""code"":""ET"",""n...","[[{""code"":""BD"",""name"":""Bangladesh""},{""code"":""B...","[[{""code"":""BY"",""name"":""Belarus""},{""code"":""AD"",...","[[{""code"":""LK"",""name"":""Sri Lanka""},{""code"":""KR..."
198,AF,Afghanistan,"[[{""code"":""DZ"",""name"":""Algeria""},{""code"":""AD"",...","[[{""code"":""AG"",""name"":""Antigua and Barbuda""},{...","[[{""code"":""BD"",""name"":""Bangladesh""},{""code"":""B...","[[{""code"":""CK"",""name"":""Cook Islands""},{""code"":...","[[{""code"":""LK"",""name"":""Sri Lanka""},{""code"":""KE..."


In [93]:
df_country_list[df_country_list.duplicated()]

,code,country,visa_required,visa_online,visa_on_arrival,visa_free_access,electronic_travel_authorisation


### **Step 2.2** Rank by year dataset

In this step i will look at the dataset to identify any cleaning, organisational steps to be taken in data cleaning step.

In [94]:
df_rank_by_year.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3950 entries, 0 to 3949
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   code             3930 non-null   object
 1   country          3950 non-null   object
 2   region           3950 non-null   object
 3   rank             3950 non-null   int64 
 4   visa_free_count  3950 non-null   int64 
 5   year             3950 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 185.3+ KB


In [95]:
df_rank_by_year.isna().sum()

,0
code,20
country,0
region,0
rank,0
visa_free_count,0
year,0


In [96]:
df_rank_by_year[df_rank_by_year['code'].isna()]

,code,country,region,rank,visa_free_count,year
2340,NaN,Namibia,AFRICA,73,78,2021
2341,NaN,Namibia,AFRICA,65,77,2020
2342,NaN,Namibia,AFRICA,69,76,2018
2343,NaN,Namibia,AFRICA,67,71,2017
2344,NaN,Namibia,AFRICA,68,70,2016
2345,NaN,Namibia,AFRICA,71,70,2015
2346,NaN,Namibia,AFRICA,60,71,2014
2347,NaN,Namibia,AFRICA,60,68,2013
2348,NaN,Namibia,AFRICA,67,67,2012
2349,NaN,Namibia,AFRICA,67,65,2011


In [97]:
df_rank_by_year.head()

,code,country,region,rank,visa_free_count,year
0,AF,Afghanistan,ASIA,116,26,2021
1,AF,Afghanistan,ASIA,106,26,2020
2,AF,Afghanistan,ASIA,106,30,2018
3,AF,Afghanistan,ASIA,104,24,2017
4,AF,Afghanistan,ASIA,104,25,2016


In [98]:
df_rank_by_year.tail()

,code,country,region,rank,visa_free_count,year
3945,PS,Palestinian Territory,MIDDLE EAST,102,37,2019
3946,PS,Palestinian Territory,MIDDLE EAST,105,37,2022
3947,PS,Palestinian Territory,MIDDLE EAST,103,38,2023
3948,PS,Palestinian Territory,MIDDLE EAST,98,40,2024
3949,PS,Palestinian Territory,MIDDLE EAST,93,39,2025


In [99]:
df_rank_by_year[df_rank_by_year.duplicated()]

,code,country,region,rank,visa_free_count,year


### **Step 2.3** Covid 19 data

This is a very complex dataset described in the repository as The Oxford Covid-19 Government Response Tracker (OxCGRT) collected information on which pandemic response measures were enacted by governments, and when. Understandably i will not be using most of the data within the dataset and will have to establish which data we will need.

In [100]:
df_covid_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202760 entries, 0 to 202759
Data columns (total 56 columns):
 #   Column                                                                           Non-Null Count   Dtype  
---  ------                                                                           --------------   -----  
 0   CountryName                                                                      202760 non-null  object 
 1   CountryCode                                                                      202760 non-null  object 
 2   RegionName                                                                       0 non-null       float64
 3   RegionCode                                                                       0 non-null       float64
 4   Jurisdiction                                                                     202760 non-null  object 
 5   Date                                                                             202760 non-null  int64  
 

In [101]:
df_covid_data.isna().sum()

,0
CountryName,0
CountryCode,0
RegionName,202760
RegionCode,202760
Jurisdiction,0
Date,0
C1M_School closing,0
C1M_Flag,74497
C2M_Workplace closing,0
C2M_Flag,68936


After quick glance at the data and the documentation for this data i think we will require following columns to attempt to tie this data with data provided with TidyTuesday challenge.

- CountryName
- CountryCode
- Date
- C8EV_International travel controls. Record restrictions on international travel. The values stored in this column range from 0 to 4. 0 - no restrictions,1 - screening arrivals
, 2 - quarantine arrivals from some or all regions
, 3 - ban arrivals from some regions
, 4 - ban on all regions or total border closure. **Note**: this records policy for foreign travellers, not citizens

##**Step 3** Cleaning the data

In this step i will clean and prepare the data to be exported to power bi for dashboard creation. First we will copy our initial datasets to keep things organised.

In [102]:
df_country_list_cleaning = df_country_list.copy()
df_rank_by_year_cleaning = df_rank_by_year.copy()

# We will use a small number of columns from the original dataset
df_covid_data_slim = df_covid_data[['CountryName', 'CountryCode', 'Date', 'C8EV_International travel controls']].copy()

### **Step 3.1** Cleaning country list dataset.

I have identified a couple of potential issues with the dataset. We have a missing country code value for Namibia. We have list of lists of dictionaries in our dataset for the visa requirements columns. This is not ideal for our purpose of working with this data in Power BI. I think the best way to approach this is to create a completely new dataframe structured to resemble a fact_table in Power BI.

**Steps to take**
1. Create a dataframe based on existing data, that will act as a fact table containg rows of data. Each row will contain country data, vista requirement data ( ex. electronic, visa free ), and related_country data. Each row will look something like this. Country Key: reference to country, Visa Key: reference to visa requirement, Related Country Key: reference to related country ( country to which visa requirements apply )
2. Create a dimension for Countries, which will store country name, country code etc.
3. Create a dimension for Visa Requirements, which will store visa requirements.

**3.1.1 Creating a fact table with a list of country codes and visa requirements.**

In [103]:
df_country_list_cleaning.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 7 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   code                             198 non-null    object
 1   country                          199 non-null    object
 2   visa_required                    199 non-null    object
 3   visa_online                      199 non-null    object
 4   visa_on_arrival                  199 non-null    object
 5   visa_free_access                 199 non-null    object
 6   electronic_travel_authorisation  199 non-null    object
dtypes: object(7)
memory usage: 11.0+ KB


**Fill in missing values.**

In [104]:
df_country_list_cleaning[df_country_list_cleaning['code'].isna()]

,code,country,visa_required,visa_online,visa_on_arrival,visa_free_access,electronic_travel_authorisation
80,NaN,Namibia,"[[{""code"":""AF"",""name"":""Afghanistan""},{""code"":""...","[[{""code"":""AZ"",""name"":""Azerbaijan""},{""code"":""C...","[[{""code"":""BD"",""name"":""Bangladesh""},{""code"":""B...","[[{""code"":""AO"",""name"":""Angola""},{""code"":""AI"",""...","[[{""code"":""LK"",""name"":""Sri Lanka""},{""code"":""SC..."


In [105]:
df_country_list_cleaning['code'] = df_country_list_cleaning['code'].fillna('NA')

**Converting data in to long form.**

The purpose of this code is to transform our dataframe in to a long format and deal with complex values stored in our visa columns. Essentially this creates new rows for each country where visa requirements are stored on each row instead of having them in dictionaries. This convert data in to a flat format and will make our life easier when working in power bi. As much as i would like to take credit for this code, i must note this code was done in **collaboration** with AI tools.  

In [106]:
def transform_dataframe(df):
    """
    Transforms the input DataFrame by un-nesting the visa data and
    creating a new long-form DataFrame.
    """

    records = []
    visa_columns = ['visa_required', 'visa_online', 'visa_on_arrival', 'visa_free_access', 'electronic_travel_authorisation']

    for _, row in df.iterrows():
        try:
            row_data = {col: ast.literal_eval(row[col]) if isinstance(row[col], str) else row[col] for col in visa_columns}
        except (ValueError, SyntaxError):
            print(f"Skipping row with parsing error.")
            continue

        for col in visa_columns:
            # Add a check to ensure the value is a list
            if isinstance(row_data[col], list):
                # Add a second loop to handle the list of lists structure
                for inner_list in row_data[col]:
                    if isinstance(inner_list, list):
                        for country_dict in inner_list:
                            if isinstance(country_dict, dict) and 'code' in country_dict:
                                new_record = {
                                    'code': row['code'],
                                    # 'country': row['country'],
                                    'related_country': country_dict['name'],
                                    'related_country_code': country_dict['code'],
                                    'visa_requirements': col
                                }
                                records.append(new_record)

    if not records:
        print("No valid records found after transformation.")
        return pd.DataFrame(columns=['code', 'related_country_code', 'visa_requirements'])

    new_df = pd.DataFrame(records)
    return new_df


In [107]:
df_visa_requirements = transform_dataframe(df_country_list_cleaning)

In [108]:
print(df_visa_requirements.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44973 entries, 0 to 44972
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   code                  44973 non-null  object
 1   related_country       44973 non-null  object
 2   related_country_code  44973 non-null  object
 3   visa_requirements     44973 non-null  object
dtypes: object(4)
memory usage: 1.4+ MB
None


In [109]:
df_visa_requirements['visa_requirements'].unique()

array(['visa_required', 'visa_online', 'visa_on_arrival',
       'visa_free_access', 'electronic_travel_authorisation'],
      dtype=object)

In [110]:
df_visa_requirements[df_visa_requirements['code'] == 'AF']

,code,related_country,related_country_code,visa_requirements
44747,AF,Algeria,DZ,visa_required
44748,AF,Andorra,AD,visa_required
44749,AF,Angola,AO,visa_required
44750,AF,Anguilla,AI,visa_required
44751,AF,Argentina,AR,visa_required
...,...,...,...,...
44968,AF,Niue,NU,visa_free_access
44969,AF,Madagascar,MG,visa_free_access
44970,AF,Sri Lanka,LK,electronic_travel_authorisation
44971,AF,Kenya,KE,electronic_travel_authorisation


**Let's save our df_visa_requirements dataframe as a fact_visa_requirements.csv**

In [111]:
df_visa_requirements = df_visa_requirements.reset_index(drop=False).rename(columns={'index': 'id'})


In [112]:
df_visa_requirements.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44973 entries, 0 to 44972
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    44973 non-null  int64 
 1   code                  44973 non-null  object
 2   related_country       44973 non-null  object
 3   related_country_code  44973 non-null  object
 4   visa_requirements     44973 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.7+ MB


**3.1.2 Creating dataframe with country names and codes.**

In [113]:
# df_countries = df_country_list_cleaning[['code', 'country']].drop_duplicates().reset_index(drop=True)

In [114]:
df_countries = df_visa_requirements[['related_country', 'related_country_code']].drop_duplicates().reset_index(drop=True)

We no longer need our related_country name in the visa requirements data. As this was only used to populate our countries dimension.

In [115]:
df_visa_requirements.drop(['related_country'], axis=1, inplace=True)

In [116]:
df_countries.rename(columns={
    "related_country": "country",
    "related_country_code": "code"
}, inplace=True)

In [117]:
df_countries['country'] = df_countries['country'].str.strip()

**Check country names for non ASCI characters using regex.**

In [118]:
# [^\x00-\x7F] matches any character outside the standard ASCII range
regex_pattern = r'[^\x00-\x7F]'

In [119]:
df_with_foreign_chars = df_countries[df_countries['country'].str.contains(regex_pattern, regex=True, na=False)]

In [120]:
unique_foreign_names = df_with_foreign_chars['country'].unique()

In [121]:
print(unique_foreign_names)

['Türkiye']


In [122]:
replacement_map = {
    'Türkiye': 'Turkey',
}

In [123]:
df_countries['country'] = df_countries['country'].replace(replacement_map)

In [124]:
df_countries[df_countries['country'] == "eSwatini"]

,country,code
218,eSwatini,SZ


In [125]:
df_countries[df_countries['code'] == "TR"]

,country,code
163,Turkey,TR


In [126]:
df_countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227 entries, 0 to 226
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   country  227 non-null    object
 1   code     227 non-null    object
dtypes: object(2)
memory usage: 3.7+ KB


In [127]:
df_countries = df_countries.reset_index(drop=False).rename(columns={'index': 'country_key'})

**3.1.3 Creating dataframe with visa requirements.**

In [128]:
df_visa_types = df_visa_requirements[['visa_requirements']].drop_duplicates().reset_index(drop=True)

In [129]:
df_visa_types.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 1 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   visa_requirements  5 non-null      object
dtypes: object(1)
memory usage: 172.0+ bytes


In [130]:
df_visa_types = df_visa_types.reset_index(drop=False).rename(columns={'index': 'visa_key'})

### **Step 3.2** Cleaning rank by year dataset.

In this step we will fill in missing values for our country code. Extract unique regions and merge them to our contries dataframe which will act as a dimension table for our fact tables.

In [131]:
df_rank_by_year_cleaning.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3950 entries, 0 to 3949
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   code             3930 non-null   object
 1   country          3950 non-null   object
 2   region           3950 non-null   object
 3   rank             3950 non-null   int64 
 4   visa_free_count  3950 non-null   int64 
 5   year             3950 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 185.3+ KB


In [132]:
df_rank_by_year_cleaning.head()

,code,country,region,rank,visa_free_count,year
0,AF,Afghanistan,ASIA,116,26,2021
1,AF,Afghanistan,ASIA,106,26,2020
2,AF,Afghanistan,ASIA,106,30,2018
3,AF,Afghanistan,ASIA,104,24,2017
4,AF,Afghanistan,ASIA,104,25,2016


Fix the missing values.

In [133]:
df_rank_by_year_cleaning[df_rank_by_year_cleaning['code'].isna()]

,code,country,region,rank,visa_free_count,year
2340,NaN,Namibia,AFRICA,73,78,2021
2341,NaN,Namibia,AFRICA,65,77,2020
2342,NaN,Namibia,AFRICA,69,76,2018
2343,NaN,Namibia,AFRICA,67,71,2017
2344,NaN,Namibia,AFRICA,68,70,2016
2345,NaN,Namibia,AFRICA,71,70,2015
2346,NaN,Namibia,AFRICA,60,71,2014
2347,NaN,Namibia,AFRICA,60,68,2013
2348,NaN,Namibia,AFRICA,67,67,2012
2349,NaN,Namibia,AFRICA,67,65,2011


In [134]:
df_rank_by_year_cleaning['region'].unique()

array(['ASIA', 'EUROPE', 'AFRICA', 'CARIBBEAN', 'AMERICAS', 'MIDDLE EAST',
       'OCEANIA'], dtype=object)

**Extract unique regions from rank_by_year dataframe.**

In [135]:
unique_regions_df = df_rank_by_year_cleaning[['region', 'country']].drop_duplicates().reset_index(drop=True)

In [136]:
unique_regions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   region   199 non-null    object
 1   country  199 non-null    object
dtypes: object(2)
memory usage: 3.2+ KB


In [137]:
unique_regions_df['country'] = unique_regions_df['country'].replace(replacement_map)

**Merge regions with our existing countries dataframe.**

In [138]:
df_countries = pd.merge(df_countries, unique_regions_df[['region', 'country']], on='country', how='left')

In [139]:
df_countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227 entries, 0 to 226
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   country_key  227 non-null    int64 
 1   country      227 non-null    object
 2   code         227 non-null    object
 3   region       199 non-null    object
dtypes: int64(1), object(3)
memory usage: 7.2+ KB


In [140]:
df_countries['region'].unique()

array(['ASIA', 'AFRICA', 'EUROPE', nan, 'AMERICAS', 'MIDDLE EAST',
       'CARIBBEAN', 'OCEANIA'], dtype=object)

Some of the countries don't have region as not being present in the source dataset. As the number of missing values not that high i think connecting to exteranl datasources is not neccesarry. Instead we will utilise AI ( like Gemini ) to create a quick list of missing regions for us.

In [141]:
df_countries[df_countries['region'].isna()][['code', 'country']]

,code,country
4,AI,Anguilla
7,AW,Aruba
13,BM,Bermuda
15,BQ,Bonaire; St. Eustatius and Saba
21,KY,Cayman Islands
30,CW,Curacao
39,FK,Falkland Islands
40,FO,Faroe Islands
44,GF,French Guiana
45,PF,French Polynesia


In [142]:
missing_region_map = {
    'AI': 'CARIBBEAN',
    'AW': 'CARIBBEAN',
    'BM': 'AMERICAS',
    'BQ': 'CARIBBEAN',
    'KY': 'CARIBBEAN',
    'CW': 'CARIBBEAN',
    'FK': 'AMERICAS',
    'FO': 'EUROPE',
    'GF': 'AMERICAS',
    'PF': 'OCEANIA',
    'FW': 'CARIBBEAN',
    'GI': 'EUROPE',
    'GL': 'AMERICAS',
    'GU': 'OCEANIA',
    'YT': 'AFRICA',
    'NC': 'OCEANIA',
    'MP': 'OCEANIA',
    'PR': 'CARIBBEAN',
    'RE': 'AFRICA',
    'AS': 'OCEANIA',
    'SH': 'AFRICA',
    'MF': 'CARIBBEAN',
    'TC': 'CARIBBEAN',
    'VG': 'CARIBBEAN',
    'VI': 'CARIBBEAN',
    'MS': 'CARIBBEAN',
    'CK': 'OCEANIA',
    'NU': 'OCEANIA'
}

In [143]:
df_countries['region'] = df_countries['region'].fillna(df_countries['code'].map(missing_region_map))

In [144]:
df_countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227 entries, 0 to 226
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   country_key  227 non-null    int64 
 1   country      227 non-null    object
 2   code         227 non-null    object
 3   region       227 non-null    object
dtypes: int64(1), object(3)
memory usage: 7.2+ KB


Lets ensure our regions are title case.

In [145]:
df_countries['region'] = df_countries['region'].str.title()

In [146]:
df_rank_by_year_cleaning.head()

,code,country,region,rank,visa_free_count,year
0,AF,Afghanistan,ASIA,116,26,2021
1,AF,Afghanistan,ASIA,106,26,2020
2,AF,Afghanistan,ASIA,106,30,2018
3,AF,Afghanistan,ASIA,104,24,2017
4,AF,Afghanistan,ASIA,104,25,2016


Clean up rank by year dataframe by removing region and country, as we will be connecting these as from our df_countries in power bi as a dimension table.

In [147]:
df_rank_by_year_cleaning.drop(['region', 'country'], axis=1, inplace=True)

In [148]:
df_rank_by_year_cleaning = df_rank_by_year_cleaning.reset_index(drop=False).rename(columns={'index': 'rank_id'})

### **Step 3.3** Cleaning Covid-19 dataset.

In [149]:
df_covid_data_slim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202760 entries, 0 to 202759
Data columns (total 4 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   CountryName                         202760 non-null  object 
 1   CountryCode                         202760 non-null  object 
 2   Date                                202760 non-null  int64  
 3   C8EV_International travel controls  202760 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 6.2+ MB


In [150]:
df_covid_data_slim['Date'] = df_covid_data_slim['Date'].astype(str).str[:4].astype(int)

In [151]:
df_covid_data_slim.rename(columns={
    'C8EV_International travel controls': 'international_travel_controls',
    'CountryName': 'country',
    'CountryCode': 'country_code',
    'Date': 'year'
}, inplace=True)

In [152]:
df_covid_data_slim.columns = df_covid_data_slim.columns.str.lower()

In [153]:
df_covid_data_slim.tail()

,country,country_code,year,international_travel_controls
202755,Zimbabwe,ZWE,2022,1.0
202756,Zimbabwe,ZWE,2022,1.0
202757,Zimbabwe,ZWE,2022,1.0
202758,Zimbabwe,ZWE,2022,1.0
202759,Zimbabwe,ZWE,2022,1.0


In [154]:
df_covid_data_slim['country'] = df_covid_data_slim['country'].str.strip()

In [155]:
correction_map = {
    'Bosnia and Herzegovina': 'Bosnia and Herzegovina',
    "Cote D'Ivoire": "Cote D'Ivoire",
    'Democratic Republic of Congo': 'Congo (Dem. Rep.)',
    'Congo': 'Congo (Rep.)',
    'Cape Verde': 'Cape Verde Islands',
    'Czech Republic': 'Czechia',
    'Faeroe Islands': 'Faroe Islands',
    'Gambia': 'The Gambia',
    'Eswatini': 'eSwatini',
    'Hong Kong': 'Hong Kong (SAR China)',
    'Kyrgyz Republic': 'Kyrgyzstan',
    'Macao': 'Macao (SAR China)',
    'Palestine': 'Palestinian Territory',
    'Russia': 'Russian Federation',
    'Slovak Republic': 'Slovakia',
    'Trinidad and Tobago': 'Trinidad and Tobago',
    'Taiwan': 'Taiwan (Chinese Taipei)',
    'United States Virgin Islands': 'US Virgin Islands'
}

In [156]:
df_covid_data_slim['country'] = df_covid_data_slim['country'].replace(correction_map)

In [157]:
df_covid_data_slim = pd.merge(df_covid_data_slim, df_countries, on='country', how='left')

In [158]:
df_covid_data_slim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202760 entries, 0 to 202759
Data columns (total 7 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   country                        202760 non-null  object 
 1   country_code                   202760 non-null  object 
 2   year                           202760 non-null  int64  
 3   international_travel_controls  202760 non-null  float64
 4   country_key                    202760 non-null  int64  
 5   code                           202760 non-null  object 
 6   region                         202760 non-null  object 
dtypes: float64(1), int64(2), object(4)
memory usage: 10.8+ MB


In [159]:
df_covid_data_slim.drop(['country','country_code', 'region', 'country_key'], axis=1, inplace=True)

In [160]:
df_covid_data_slim = df_covid_data_slim.reset_index(drop=False).rename(columns={'index': 'id'})

In [161]:
df_covid_data_slim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202760 entries, 0 to 202759
Data columns (total 4 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   id                             202760 non-null  int64  
 1   year                           202760 non-null  int64  
 2   international_travel_controls  202760 non-null  float64
 3   code                           202760 non-null  object 
dtypes: float64(1), int64(2), object(1)
memory usage: 6.2+ MB


**Let's create a dimension table for our international travel controls**

In [162]:
dim_travel_controls = pd.DataFrame({
    'control_key': [0, 1, 2, 3, 4],
    'control_description': [
        'No restrictions',
        'Screening arrivals',
        'Quarantine arrivals from some or all regions',
        'Ban arrivals from some regions',
        'Ban on all regions or total border closure'
    ]
})

## **Finally lets save datasets to CSV to be exported in to Power BI**

In [163]:
df_countries.to_csv('dim_countries.csv', index=False)

In [164]:
df_visa_types.to_csv('dim_requirements.csv', index=False)

In [165]:
df_visa_requirements.to_csv('fact_visa_requirements.csv', index=False)

In [166]:
df_rank_by_year_cleaning.to_csv('fact_rank.csv', index=False)

In [167]:
df_covid_data_slim.to_csv('fact_covid.csv', index=False)

In [168]:
dim_travel_controls.to_csv('dim_travel_controls.csv', index=False)